In [1]:
import numpy as np
import pandas as pd
import streamlit as st
import csv
from collections import defaultdict
from numpy.random import default_rng as rng

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
from os import path
from matplotlib.pyplot import figure
if 'Sarabun' not in [x.name for x in matplotlib.font_manager.fontManager.ttflist]:
    matplotlib.font_manager.fontManager.addfont('Sarabun-Regular.ttf')
matplotlib.rc('font', family='Sarabun')

from rsi_modules import *

mm3_url         = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=1475905714"
publication_url = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=2125519370"
theses_url      = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=609352385"

F = pd.read_csv(mm3_url)
publication = pd.read_csv(publication_url).fillna("")
theses = pd.read_csv(theses_url).fillna("")
theses = theses.drop_duplicates(subset=['รหัส (นักศึกษา)'], keep='last')

In [2]:
publication,df = get_publication(publication)
df = df.reset_index().rename(columns={'index':'ปี'})
# df.to_csv('publication.csv')
# st.write(df)

2026-02-11 15:19:43.870 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-11 15:19:44.007 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-11 15:19:44.157 
  command:

    streamlit run /home/vscode/.local/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2026-02-11 15:19:44.158 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-11 15:19:44.160 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [3]:
def classify_publication(publication):  

    NAMES = []
    
    for n in publication.columns:
        if 'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ' in n:
            NAMES.append(n) 

    for t in publication.columns:
        if 'วารสารไทย (TCI1-TCI2)' in t: publication.rename(columns={t:'วารสารไทย'}, inplace=True)
        elif 'วารสารนานาชาติ (Q1-Q4 หรืออื่นๆ)' in t: publication.rename(columns={t:'วารสารนานาชาติ'}, inplace=True)
        elif 'ประชุมวิชาการระดับนานาชาติ' in t: publication.rename(columns={t:'ประชุมวิชาการระดับนานาชาติ'}, inplace=True)
        elif 'ประชุมวิชาการระดับชาติ' in t: publication.rename(columns={t:'ประชุมวิชาการระดับชาติ'}, inplace=True)

    name_paper = defaultdict(list)
    for name in NAMES:
        papers = []
        for i,(role,TCI,Q,conf_int,conf_nat) in enumerate(zip(publication[name],
                                                            publication['วารสารไทย'],
                                                            publication['วารสารนานาชาติ'],
                                                            publication['ประชุมวิชาการระดับนานาชาติ'],
                                                            publication['ประชุมวิชาการระดับชาติ'])):
            if role in ['First Author', 'Corresponding Author']:
                papers+=[TCI+Q+conf_int+conf_nat]
        name_paper[name.replace('ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [','').replace(']','')] = papers

    categories = ['Q1','Q2','Q3','Q4','TCI1','TCI2','อื่นๆ']
    blanktable = pd.DataFrame(np.zeros((len(name_paper.keys()), len(categories)),dtype=int), columns=categories)
    blanktable['name']=name_paper.keys()


    for idx, (name, papers) in enumerate(name_paper.items()):
        for paper in papers:
            for cat in categories:
                if cat in paper:
                    blanktable.loc[idx, cat] += 1
    
    # Reorder columns with 'name' first
    blanktable = blanktable[['name'] + [col for col in blanktable.columns if col != 'name']]
    
    # blanktable.to_csv('publication_classified.csv')

    return blanktable

In [ ]:
def get_publications_by_author(publication):
    """
    Create a dictionary where:
    - Key: author name (First Author or Corresponding Author)
    - Value: list of tuples (year, title, publication_type)
    """
    
    # Find author columns
    NAMES = []
    for n in publication.columns:
        if 'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ' in n:
            NAMES.append(n)
    
    # Rename journal/conference columns for easier processing
    pub_copy = publication.copy()
    for t in pub_copy.columns:
        if 'วารสารไทย (TCI1-TCI2)' in t: 
            pub_copy.rename(columns={t:'วารสารไทย'}, inplace=True)
        elif 'วารสารนานาชาติ (Q1-Q4 หรืออื่นๆ)' in t: 
            pub_copy.rename(columns={t:'วารสารนานาชาติ'}, inplace=True)
        elif 'ประชุมวิชาการระดับนานาชาติ' in t: 
            pub_copy.rename(columns={t:'ประชุมวิชาการระดับนานาชาติ'}, inplace=True)
        elif 'ประชุมวิชาการระดับชาติ' in t: 
            pub_copy.rename(columns={t:'ประชุมวิชาการระดับชาติ'}, inplace=True)
    
    author_publications = defaultdict(list)
    
    # Get year and title columns
    year_col = 'ปีที่ตีพิมพ์ พศ. (คศ.)'
    title_col = 'ชื่อเรื่อง'
    
    # Process each row
    for idx, row in pub_copy.iterrows():
        year = row.get(year_col, '')
        title = row.get(title_col, '')
        
        # Determine publication type (Q or TCI category)
        pub_type = 'อื่นๆ'
        for category in ['Q1', 'Q2', 'Q3', 'Q4', 'TCI1', 'TCI2']:
            combined_text = (str(row.get('วารสารไทย', '')) + 
                           str(row.get('วารสารนานาชาติ', '')) +
                           str(row.get('ประชุมวิชาการระดับนานาชาติ', '')) +
                           str(row.get('ประชุมวิชาการระดับชาติ', '')))
            if category in combined_text:
                pub_type = category
                break
        
        # Process each author
        for name_col in NAMES:
            role = row.get(name_col, '')
            if role in ['First Author', 'Corresponding Author']:
                # Extract author name from column name
                author_name = name_col.replace('ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [', '').replace(']', '')
                # Add publication tuple (year, title, type)
                author_publications[author_name].append((year, title, pub_type))
    
    return dict(author_publications)


# Export author_pubs to CSV
def export_author_publications_to_csv(author_pubs, filename='author_publications.csv'):
    """
    Export author publications dictionary to CSV file
    Format: Author, Year, Publication Type, Title
    """
    rows = []
    for author, publications in author_pubs.items():
        for year, pub_type, title in publications:
            rows.append({
                'Author': author,
                'Year': year,
                'Publication Type': pub_type,
                'Title': title
            })
    
    df = pd.DataFrame(rows)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Exported {len(rows)} publications to {filename}")
    return df

# Test the function
author_pubs = get_publications_by_author(publication)
df_export = export_author_publications_to_csv(author_pubs, 'author_publications.csv')


{'ดร.วิษณุ นิตยธรรมกุล': [('2568 (2025)',
   'Using Virtual Reality and Conversational AI to Enhance High School Students’ Intentions to Pursue Computer Science: Insights from the CS RMUTT Virtual Tour',
   'Q2'),
  ('2568 (2025)',
   'Development of Low-Cost PLC Training Kit for Enhancing Industry-Relevant Skills and Certification: PLC-CertX',
   'Q3'),
  ('2568 (2025)',
   'Design of the System Architecture of Competency-Based MOOC Using Microlearning Objects to Facilitate Upskilling and Reskilling for Industrial Workforce: Digital MicroLearn',
   'อื่นๆ')],
 'ราษฏร์ บุญญา': [('2565 (2022)',
   'การพัฒนาบทเรียนอิเล็กทรอนิกส์เพื่อส่งเสริมทักษะการใช้ภาษามือไทยสำหรับนักศึกษาระดับปริญญาตรี',
   'TCI2'),
  ('2566 (2023)',
   'The Development of E-Learning on Topic of Impact of Foreign Education to Thai Deaf Education for Undergraduate Students',
   'TCI1')],
 'ดร.ปกรณ์กิตติ์ ม่วงประสิทธิ์': [('2564 (2021)',
   'ปกรณ์กิตติ์ ม่วงประสิทธิ์, ศิริณา จิตต์จรัส และอรอุษา ปุณยบุรณะ. (2564). ศาสตร

In [5]:
author_pubs

{'ดร.วิษณุ นิตยธรรมกุล': [('2568 (2025)',
   'Using Virtual Reality and Conversational AI to Enhance High School Students’ Intentions to Pursue Computer Science: Insights from the CS RMUTT Virtual Tour',
   'Q2'),
  ('2568 (2025)',
   'Development of Low-Cost PLC Training Kit for Enhancing Industry-Relevant Skills and Certification: PLC-CertX',
   'Q3'),
  ('2568 (2025)',
   'Design of the System Architecture of Competency-Based MOOC Using Microlearning Objects to Facilitate Upskilling and Reskilling for Industrial Workforce: Digital MicroLearn',
   'อื่นๆ')],
 'ราษฏร์ บุญญา': [('2565 (2022)',
   'การพัฒนาบทเรียนอิเล็กทรอนิกส์เพื่อส่งเสริมทักษะการใช้ภาษามือไทยสำหรับนักศึกษาระดับปริญญาตรี',
   'TCI2'),
  ('2566 (2023)',
   'The Development of E-Learning on Topic of Impact of Foreign Education to Thai Deaf Education for Undergraduate Students',
   'TCI1')],
 'ดร.ปกรณ์กิตติ์ ม่วงประสิทธิ์': [('2564 (2021)',
   'ปกรณ์กิตติ์ ม่วงประสิทธิ์, ศิริณา จิตต์จรัส และอรอุษา ปุณยบุรณะ. (2564). ศาสตร

In [ ]:


# Export to CSV
df_export.head()


Exported 18 publications to author_publications.csv


,Author,Year,Publication Type,Title
0,ดร.วิษณุ นิตยธรรมกุล,2568 (2025),Using Virtual Reality and Conversational AI to...,Q2
1,ดร.วิษณุ นิตยธรรมกุล,2568 (2025),Development of Low-Cost PLC Training Kit for E...,Q3
2,ดร.วิษณุ นิตยธรรมกุล,2568 (2025),Design of the System Architecture of Competenc...,อื่นๆ
3,ราษฏร์ บุญญา,2565 (2022),การพัฒนาบทเรียนอิเล็กทรอนิกส์เพื่อส่งเสริมทักษ...,TCI2
4,ราษฏร์ บุญญา,2566 (2023),The Development of E-Learning on Topic of Impa...,TCI1


In [ ]:
st.title('ภาระงานของคณาจารย์ในสถาบันราชสุดาปีการศึกษา 2568')

st.subheader("1. ชั่วโมงสอนและจำนวนวิชาที่รับผิดชอบ")

academic_year = st.radio(
    "เลือกเทอมและปีการศึกษา",
    ["2568/1", "2568/2"],
)

if academic_year == "2568/1":
    start_date = '8/5/2025'
    end_date   = '9/19/2025'
elif academic_year == "2568/2":
    start_date = '12/12/2025'
    end_date   = '12/30/2025'

f = F[F['Timestamp'].between(start_date, end_date)]
t1,t2 = report_mm3(f)

st.subheader("2. ผลงานตีพิมพ์ทางวิชาการในแต่ละปี")
publication,df = get_publication(publication)
df = df.reset_index().rename(columns={'index':'ปี'})
st.write(df)

st.subheader("3. ภาระงานควบคุมวิทยานิพนธ์")
B = pd.DataFrame(get_thesis_workload_new(theses)).T
st.write(B)

st.subheader("4. ภาพรวมความก้าวหน้าวิทยานิพนธ์ของนักศึกษา")
N = get_thesis_progression_summary(theses)
A = pd.concat(N, axis=1)
st.write(A)
st.write("หมายเหตุ:")
st.write("MA  = หส.ปริญญาโท สาขาคุณภาพชชีวิตคนพิการ")
st.write("MEd = หส.ปริญญาโท สาขาการศึกษาพิเศษ")
st.write("PhD = หส.ปรัชญาดุษฎีบัณฑิต สาขาคุณภาพชีวิตคนพิการ")

2025-12-24 13:56:05.508 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.340 
  command:

    streamlit run c:\Users\User\miniconda3\envs\rsi_dashboard\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-24 13:56:06.341 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.342 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.343 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.344 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.344 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.345 Thre

In [2]:
N = get_thesis_progression_summary(theses)
A = pd.concat(N, axis=1)

In [3]:
st.write(A)

2025-12-17 17:05:31.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-17 17:05:31.468 
  command:

    streamlit run c:\Users\User\miniconda3\envs\rsi_dashboard\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-17 17:05:31.468 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-17 17:05:31.470 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
